http://blog.plataformatec.com.br/2011/03/cognitive-load-and-the-importance-of-naming/

A tendency of science is to describe increasingly complex ideas in more compact terms.  Eventually devolving into an alphabet soup of acronyms where creativity meets its darkest and lowest tendencies.

But, naming is hard right?  Or rather naming is unnatural.

On a computer, in the throws of creativity, the programming environment should elevate this activity.  Translating `phrases` into valid code may stymie or distract from the creative process.

One way to reduce cognitive load on naming is name things less.  _Personally, I have found that functional programming significantly reduces naming._

Another to name things is easier is to make the translation of natural language into names easier.  The naming system much naturally evolve with the work.

For this reason, this blog post considers `Phrase`s.  `Phrase`s are modules created from strings meant to manage the state of a `Phrase`.  `Phrase`s treat natural language statements as python objects.  They have str, repr, and markdown displays.  Representing natural language as a unified code object will allow its changes to be captured in the revision of code over time.

In [92]:
%reload_ext deathbeds.__Custom_display_formatting
import IPython, abc, types, sys, importlib.machinery, types, abc, string, jinja2, stringcase
ip = get_ipython()

A `Phrase` is a `types.ModuleType` the represents the state of a string typically expressed in natural language.  A `Phrase` has no inheritence, but may maintain state. 

* The phrase is converted to a sentence.
* Installed as a `sys.modules` key.
* Installed into __main__ so that the phrases appear with tab completion.

In [121]:
class Phrase(types.ModuleType):    
    def __init__(self, name, doc=None, **dict):
        super().__init__(to_sentence(name), doc)
        # Assign the phrase to main so it recieves valid code prediction.
        setattr(__import__('__main__'), self.__name__, self)
        self.__dict__.update(dict)  

`types.ModuleType` was chosen because it provides an isolated namespace and the import system is a canonical way to access `Phrase` strings.

A `Phrase` has a custom `__call__` method that is syntactic sugar for updating the state of a module.

In [122]:
def __call__(self, doc=None, **objects):         
    """Set the docstring or update the module attributes
    >>> Phrase()"""
    doc and objects.update(__doc__=doc)
    return self.__dict__.update(objects) or self
Phrase.__call__ = __call__

For extra flare, lets use the rshift operator to update the docstring.

In [144]:
def __rshift__(self, object):
    if isinstance(object, dict): self.__dict__.update(object)
    if isinstance(object, str): self.__doc__ = object
    return self

Phrase.__rshift__ = __rshift__

The `Phrase.__name__` is the string representation which is generally a natural language string.

In [145]:
Phrase.__str__ = lambda self: self.__name__

The `repr` provides a deeper representation and fallbacks to `Phrase.__name__`.

In [146]:
Phrase.__repr__ = lambda self: self.__doc__ or self.__name__

Phrase.__doc__ and Phrase.__name__ supply an interactive markdown repr for easy reuse.

In [147]:
Phrase._repr_markdown_ = lambda self: self.__doc__ or f"__{str(self)}__"

There should be a means to combine `Phrases`.  _At this point, Phrases need to get used._

In [152]:
Phrase.__and__ = lambda self, object: Phrases((self, object))

In [153]:
class Phrases(__import__('collections').UserList):
    def __str__(self): return ''.join(
        ('' if data is self[0] else 
         ', ' if data is not self[-1] else
         ' and ') + str(data) for data in self)
    
    @property
    def __doc__(self): return '\n\n---\n\n'.join(object._repr_markdown_() for object in self)

    _repr_markdown_ = Phrase._repr_markdown_
    

In [154]:
def to_sentence(string):
    lower, string = string[0].islower(), stringcase.sentencecase(string)
    return [str.upper, str.lower][lower](string[0]) + string[1:]

## Importing phrases.

`Phrases` are __objects with views defined by a string__.  I ❤️ hacking the import system.  For `Phrase`s  we will rely on the `__import__` function to load phrases as strings.

`PhrasesBase` is a stateless contextmanager to "discover" a `importlib.machinery.ModuleSpec` the will become a `Phrase` module.

In [37]:
class PhrasesBase(abc.ABCMeta):
    def __enter__(cls): sys.meta_path.append(phrases)
    def __exit__(cls, *x): sys.meta_path = [x for x in sys.meta_path if not issubclass(x, phrases)]
    def find_spec(cls, name, *args, **kwargs): return importlib.machinery.ModuleSpec(name, phrases)

`phrases` is a contextmanager to `__import__` the `Phrase`.

In [157]:
class phrases(metaclass=PhrasesBase):
    create_module = staticmethod(lambda spec: Phrase(spec.name, globals=False))
    exec_module = staticmethod(lambda module: module)
    is_package = staticmethod(lambda _: True)

In [158]:
def test_importing_with_ipython_syntax():
    with phrases:
        ;__import__ This experiment
        ;str was
        ;__import__ a success

Templating should be added.

In [15]:
{
    '{{}}': jinja2.Template,
    '{}': str.format,
    '%': str.__mod__,
    '$': string.Template
}

{'{{}}': jinja2.environment.Template,
 '{}': <method 'format' of 'str' objects>,
 '%': <slot wrapper '__mod__' of 'str' objects>,
 '$': string.Template}